In [ ]:
from clearml import Task
import configparser
import shutil
import json
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent))
from task_processing import remote2local, save_tasks, LocalTask

In [ ]:
def get_credentials(dir_, file):
    if file is not None:
        config = configparser.ConfigParser()
        config.read(f'{dir_}/{file}.conf')

        # Get the values of the different settings
        api_server = config.get('settings', 'api_server')
        web_server = config.get('settings', 'web_server')
        files_server = config.get('settings', 'files_server')

        # Parse the credentials JSON string
        credentials_json = config.get('settings', 'credentials')
        credentials = json.loads(credentials_json)
        access_key = credentials['access_key']
        secret_key = credentials['secret_key']
        return api_server, web_server, files_server, access_key, secret_key
    else:
        return None, None, None, None, None

In [ ]:
configuration = {
    'config_dir': '../configs',
    'server': 'local_server',
    'project_name': "AI Fairness",
    'status': ['completed'],
    'type': ['training'],
    'system_tags': ['-archived'],
    'tags': [],
    'task_ids': None,
    'task_name': None
}

In [ ]:
api_server, web_server, files_server, access_key, secret_key = get_credentials(configuration['config_dir'], configuration['server'])
if api_server is not None:
    Task.set_credentials(
        api_host=api_server, web_host=web_server, files_host=files_server,
        key=access_key, secret=secret_key
     )
else:
    print('Use default clearml.conf')

In [ ]:
task_filter={
    # filter out archived Tasks
    'system_tags': configuration['system_tags'],
    # only completed & published Tasks
    'status': configuration['status'],
    # only training type Tasks
    'type': configuration['type']
}
tags = configuration['tags']
task_ids = configuration['task_ids']
project_name = configuration['project_name']
task_name = configuration['task_name']

task_list_ = Task.get_tasks(
    task_ids=task_ids,  # type Optional[Sequence[str]]
    project_name=project_name,  # Optional[str]
    task_name=task_name,  # Optional[str]
    task_filter=task_filter,
    tags=tags# Optional[Dict]
)

In [ ]:
len(task_list_)

In [ ]:
task_list = remote2local(task_list_)

In [ ]:
shutil.copytree('../saved_tasks', '../saved_tasks_backup')

In [ ]:
save_tasks(task_list, add_imported_tag=True)